Lab7
By: Hayden Donofrio, et al.


In [34]:
#lets prepare the data. Reading from the train.txt file
#make sure to concatenate the test.txt and train.txt files together so we can do our own split
import nltk
nltk.download('punkt')
import numpy as np
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
X = []
y = []
data = []
stop_words = set(stopwords.words('english')) 
with open("test.ft.txt", "r") as f:
    for line in f:
        if line.split()[0] == "__label__2": #positive label
            y.append(np.array([1,0]))
        else:
            y.append(np.array([0,1]))
        data.append(line[1:])

NUM_TOP_WORDS = None
MAX_ART_LEN = 1000 
tokenizer = Tokenizer(num_words=NUM_TOP_WORDS)
tokenizer.fit_on_texts(data)
sequences = tokenizer.texts_to_sequences(data)

word_index = tokenizer.word_index
NUM_TOP_WORDS = len(word_index) if NUM_TOP_WORDS==None else NUM_TOP_WORDS
top_words = min((len(word_index),NUM_TOP_WORDS))
print('Found %s unique tokens. Distilled to %d top words.' % (len(word_index),top_words))

X = pad_sequences(sequences, maxlen=MAX_ART_LEN)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/haydendonofrio/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Found 291458 unique tokens. Distilled to 291458 top words.


In [35]:
#X[0:100]

In [38]:
#lets see how evenly spread our ys are to help select a metric
# since they are one hot encoded we can just some along each column
y = np.array(y)

In [39]:
print("positive class #: ", sum(y[:,0]), " Negative class #: ", sum(y[:,1]))

positive class #:  200000  Negative class #:  200000


We can see that the classes are evenly split! We are not specifically targeting a positive class over a negative class therefore accuracy would not be a bad metric for this dataset!

We have a very large amount of data for each class. An 80/20 split should be sufficent since we have over 100,000 items for each class

In [41]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train_ohe, y_test_ohe = train_test_split(X, y, test_size=0.2)

In [43]:
#now lets load the pre-trained keras embedding
!ls "glove/" 
EMBED_SIZE = 300
# the embed size should match the file you load glove from
embeddings_index = {}
f = open('glove/glove.6B.300d.txt')
# save key/array pairs of the embeddings
#  the key of the dictionary is the word, the array is the embedding
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))
embedding_matrix = np.zeros((len(word_index) + 1, EMBED_SIZE))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

print(embedding_matrix.shape)

ls: large_data/glove/: No such file or directory
Found 400000 word vectors.
(291459, 300)


In [44]:
from keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            EMBED_SIZE,
                            weights=[embedding_matrix],
                            input_length=MAX_ART_LEN,
                            trainable=False)

In [47]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

rnn = Sequential()
rnn.add(embedding_layer)
rnn.add(LSTM(300,dropout=0.2, recurrent_dropout=0.2))
rnn.add(Dense(2, activation='sigmoid'))
rnn.compile(loss='categorical_crossentropy', 
              optimizer='rmsprop', 
              metrics=['accuracy'])
print(rnn.summary())


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1000, 300)         87437700  
_________________________________________________________________
lstm_3 (LSTM)                (None, 300)               721200    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 602       
Total params: 88,159,502
Trainable params: 721,802
Non-trainable params: 87,437,700
_________________________________________________________________
None


In [50]:
#change this [0:50] to a larger size, I made it smaller so we can focus on graphing history and then moving it up later
history = rnn.fit(X_train[0:50], y_train_ohe[0:50], validation_data=(X_test[0:5000], y_test_ohe[0:5000]), epochs=3, batch_size=64)

Train on 50 samples, validate on 5000 samples
Epoch 1/3
50/50 [==============================] - 379s 8s/step - loss: 0.7580 - accuracy: 0.5600 - val_loss: 0.6577 - val_accuracy: 0.6288
Epoch 2/3
50/50 [==============================] - 447s 9s/step - loss: 0.5974 - accuracy: 0.8000 - val_loss: 0.6548 - val_accuracy: 0.6194
Epoch 3/3
50/50 [==============================] - 563s 11s/step - loss: 0.5544 - accuracy: 0.8600 - val_loss: 0.6500 - val_accuracy: 0.6184


In [53]:
#this is the gru
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, GRU

gru = Sequential()
gru.add(embedding_layer)
gru.add(GRU(300,activation='tanh',recurrent_activation='sigmoid',dropout=0.2, recurrent_dropout=0.2))
gru.add(Dense(2, activation='sigmoid'))
gru.compile(loss='categorical_crossentropy', 
              optimizer='rmsprop', 
              metrics=['accuracy'])
print(gru.summary())


Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1000, 300)         87437700  
_________________________________________________________________
gru_1 (GRU)                  (None, 300)               540900    
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 602       
Total params: 87,979,202
Trainable params: 541,502
Non-trainable params: 87,437,700
_________________________________________________________________
None


In [ ]:
gru_history = rnn.fit(X_train[0:50], y_train_ohe[0:50], validation_data=(X_test[0:5000], y_test_ohe[0:5000]), epochs=3, batch_size=64)